In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [2]:
# show keyspaces
r = session.execute('select * from system_schema.keyspaces')
df = pd.DataFrame(r)
df

,keyspace_name,durable_writes,replication
0,system_auth,True,"[class, replication_factor]"
1,system_schema,True,[class]
2,system_distributed,True,"[class, replication_factor]"
3,system,True,[class]
4,system_traces,True,"[class, replication_factor]"


In [3]:
# create keyspace
session.execute("""
CREATE KEYSPACE IF NOT EXISTS datahungry
  WITH REPLICATION = { 
   'class' : 'SimpleStrategy', 
   'replication_factor' : 3
  };
""").one()


In [4]:
# use keyspace
session.set_keyspace('datahungry')

In [5]:
# create table
session.execute("""
CREATE TABLE user_profile(
    ssn text,
    username text,
    name text,
    sex text,
    mail text,
    birthdate date,
    address text,
    residence text,
    blood_group text,
    job text,
    company text,
    PRIMARY KEY (ssn)
   );
""").one()


In [6]:
# gen data
import pandas as pd
from faker import Faker
fake = Faker()
# https://github.com/joke2k/faker/tree/master/faker/providers/ssn

n = 1000
data = [fake.profile() for _ in range(n)]

df = pd.json_normalize(data)
df

,job,company,ssn,residence,current_location,blood_group,website,username,name,sex,address,mail,birthdate
0,Marketing executive,"Koch, Johnson and Armstrong",237-43-7807,"97107 Rivera Spring\nOwensmouth, NC 29553","(-81.608262, -127.661585)",B+,[http://wong.com/],ryanstevenson,Nathan Bates,M,"03126 Patel Park\nJonesshire, PR 31592",gregory64@yahoo.com,1954-05-31
1,Warden/ranger,Dixon and Sons,791-76-1920,"236 Williams Corners\nEast Christophershire, F...","(28.9453575, -88.693813)",O+,"[http://gibson.com/, https://www.smith.com/, h...",wilsonrachel,Benjamin Jackson,M,"7529 Ronald Centers Suite 686\nBauerfort, NJ 9...",carrstephanie@yahoo.com,1986-11-01
2,"Surveyor, minerals",Alvarez-Carr,846-32-3421,23623 Jenkins Burg Suite 288\nSouth Dorothytow...,"(-34.0100665, 152.925021)",A-,"[http://oliver.com/, https://www.gregory.com/,...",jmoreno,Timothy Wilson,M,"34723 Laura Walks Apt. 649\nWrightside, WV 86741",tmitchell@hotmail.com,1916-08-09
3,Cabin crew,Knight-Porter,110-35-4326,61697 Parker Corners Apt. 156\nNorth Jeffreymo...,"(-17.1779445, 58.981374)",O-,"[http://www.johnson.com/, http://www.owens.com/]",nhaley,Marc Beltran,M,"805 Joshua Place Suite 463\nWest Keithside, VA...",qjackson@yahoo.com,1992-11-24
4,"Civil engineer, consulting",Perez PLC,266-34-6878,66454 Bautista Parkways Suite 885\nEast Andrea...,"(-85.3464525, -93.926236)",O+,"[http://www.lee.biz/, https://owen.info/, http...",laurenmartin,Clinton Zuniga,M,"62384 Michael Turnpike\nEdwardshire, MH 29160",martinezjames@hotmail.com,1948-12-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Web designer,"Wright, Wagner and Jones",610-16-2786,"74611 Holmes Pass\nWest Davidfurt, WA 75658","(76.0309235, -46.333331)",B+,[http://www.rivera-booth.com/],milesbradley,Nathan Gonzalez,M,"PSC 0251, Box 8845\nAPO AA 93022",richard80@yahoo.com,1947-09-27
996,"Nurse, children's",Wood Inc,501-07-3850,"425 Murphy Village Apt. 849\nNew Donnaborough,...","(9.9666315, 85.462617)",B-,"[http://www.johnson.com/, https://www.castro.c...",itodd,Julie Cruz,F,"015 Kayla Hills\nSimsville, UT 21494",johnhopkins@gmail.com,1994-03-08
997,Chief Executive Officer,Franco-Garcia,514-88-3464,"00179 Potter Square Apt. 693\nNorth Cynthia, N...","(-79.984090, 149.458436)",AB+,"[http://hernandez.net/, http://www.santos.org/...",alexishampton,Dana Williams,F,"091 Reyes Vista Suite 327\nHursthaven, DC 43472",jamescoleman@yahoo.com,1992-07-02
998,Telecommunications researcher,"Smith, Jimenez and Brown",404-23-4974,"53218 Kaylee Mountains\nPort Diana, KY 00937","(-62.337590, 46.453310)",O-,[http://ware.com/],uguzman,Erik Williams,M,"062 William Keys\nEast Brianburgh, OH 27971",nicholsalexandria@gmail.com,2011-02-26


In [9]:
# insert data
col_list = ['ssn', 'username', 'name', 'sex', 'mail', 'birthdate', 'address', 'residence', 'blood_group', 'job', 'company']
df = df[col_list]

col_str = ", ".join(col_list)
question_str = ", ".join(["?"]*len(col_list))

# "INSERT INTO user_profile(ssn, username, name, sex, mail, birthdate, address, residence, blood_group, job, company) VALUES (?,?,?,?,?,?,?,?,?,?,?)"
query = f"INSERT INTO user_profile({col_str}) VALUES ({question_str})"
prepared = session.prepare(query)

# execute / execute_async
for v in tqdm(df.values):
    session.execute(prepared, v)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [10]:
# get data
r = session.execute("SELECT * FROM user_profile")
df = pd.DataFrame(r)
df

,ssn,address,birthdate,blood_group,company,job,mail,name,residence,sex,username
0,730-48-7092,"1103 Julie Locks\nRiveraland, VI 91786",2002-07-08,B-,"Salas, Anderson and Odom","Designer, exhibition/display",karen67@yahoo.com,Brittney Briggs,"5377 Gary Parkway\nSuzannemouth, LA 03975",F,kenneth88
1,242-31-1042,"2199 Johnston Drives\nNorth Garrettstad, SD 80852",1960-06-13,B+,Watson-Smith,"Investment banker, corporate",kristopherhardy@yahoo.com,Chase Hunt,"58862 Erica Light Suite 224\nEast Jeffport, NE...",M,wvillegas
2,813-80-5221,"68821 Brown Ways Apt. 597\nWest Jeanne, HI 26966",1983-06-08,A-,"Ramirez, Bowers and Green","Horticulturist, amenity",ulawson@gmail.com,Hannah Reyes,"9664 Stone Station Apt. 201\nCalhounmouth, CO ...",F,maryscott
3,859-46-5879,"9146 Brittany View\nNorth Timothy, MI 62731",1931-03-02,AB+,Heath Group,Broadcast journalist,dylansmith@yahoo.com,Alejandro Sherman,"2703 Taylor Cliffs Suite 996\nZunigaberg, NY 7...",M,dmoore
4,791-34-6764,7640 Christopher Mountains Apt. 960\nTaylorpor...,1924-11-15,O+,"Williams, Lewis and Raymond",Microbiologist,andrea11@gmail.com,Stephen Price,"16831 Thomas Mews\nWest Regina, PA 21409",M,jimenezevelyn
...,...,...,...,...,...,...,...,...,...,...,...
995,410-53-7417,"5702 Fowler Pines Apt. 621\nNorth Lisa, MA 85788",1998-08-11,B+,"Proctor, Smith and Martin",Emergency planning/management officer,garrettellis@gmail.com,Erica Williams,"948 Dean Knoll Apt. 062\nEast Rachel, MA 17883",F,naguilar
996,597-27-2196,"3371 Knight Hill Suite 055\nDavidhaven, MN 59998",1937-07-30,B+,Lee Inc,Press photographer,jessicasuarez@hotmail.com,Amanda Lynch,"164 Krystal Shore\nNew Stephanie, AL 53226",F,cmiles
997,705-50-3116,"166 Campbell Shores Apt. 638\nKeithshire, MA 9...",1995-02-09,AB-,"Hawkins, Dudley and Murphy",Comptroller,charleshamilton@gmail.com,Robert Hoffman,"53519 Patricia Parks\nPort Vanessa, NC 62433",M,brandihall
998,478-29-1506,"849 Chad Curve Suite 539\nWest Markchester, NY...",2021-10-26,AB-,White Group,Financial trader,kristinphillips@gmail.com,Raymond Ramsey,"49928 Gary Highway\nEast William, NH 87995",M,sean76
